# 演習 1: マルコフ決定過程 (MDP)


## 注意
全ての回答はこのノートブック内に書いてください．他のファイルを書いたり編集したりする必要はありません．

**セル間には依存関係があるため，全てのブロックを実行してください**
--------------------------

In [ ]:
import numpy as np, numpy.random as nr, gym
import matplotlib.pyplot as plt
%matplotlib inline
np.set_printoptions(precision=3)

### 試行ベース (表形式) Q-Learning

モデルがMDPであることが必要な方策反復 (Value Iteration) や 価値反復 (Policy Iteration) は，環境がブラックボックスな物理シミュレータとして与えられ全ての場合の変遷を得られないような場合などに限界があります．

このような環境でも，試行ベースの Q learning により学習を行うことができます．


この演習ではクローラロボットの制御を学びます．まずは完全にランダムな行動によりロボットがどのように動くかを見て，Gym に慣れましょう！

In [ ]:
from crawler_env import CrawlingRobotEnv

env = CrawlingRobotEnv()

print("この Gym 環境の観察空間と行動空間を見てみましょう．")
print("-----------------------------------------------------------------------------")
print("行動空間:", env.action_space)
print("これは可能な %i 種の行動についての離散空間です．" % env.action_space.n)
print("各行動は各関節の角度を増加，減少させることに対応します．")
print("行動空間でランダムに試行をすることができます．:", env.action_space.sample())
print("2回目の試行:", env.action_space.sample())
print("3回目の試行", env.action_space.sample())
print("観察空間:", env.observation_space, ", これは 9x13 のグリッドであることを意味します.")
print("ロボットの2つの関節の角度の離散表現です．")

In [ ]:
env = CrawlingRobotEnv(
    render=True, # turn render mode on to visualize random motion
)

# standard procedure for interfacing with a Gym environment
cur_state = env.reset() # reset environment and get initial state
ret = 0.
done = False
i = 0
while not done:
    action = env.action_space.sample() # sample an action randomly
    next_state, reward, done, info = env.step(action)
    ret += reward
    cur_state = next_state
    i += 1
    if i == 1500:
        break # for the purpose of this visualization, let's only run for 1500 steps
        # also note the GUI won't close automatically

In [ ]:
# you can close the visualization GUI with the following method 
env.close_gui()

ランダム制御も時々は成功しますが，とてもうまくいくわけではないことがわかります．
より良い戦略を得るため，ε-グリーディー探索による表形式 Q-learning を少しずつ実装して行きましょう．


In [ ]:
from collections import defaultdict
import random

# dictionary that maps from state, s, to a numpy array of Q values [Q(s, a_1), Q(s, a_2) ... Q(s, a_n)]
#   and everything is initialized to 0.
q_vals = defaultdict(lambda: np.array([0. for _ in range(env.action_space.n)]))

print("状態 (0, 0) のQ値: %s" % q_vals[(0, 0)], "これはそれぞれの行動についてのQ値のリストになっています．")
print("つまり，状態 (1, 2) において行動 3 を行う時の Q 値 i.e. Q((1,2), 3) は，このようにアクセスできます q_vals[(1,2)][3]:", q_vals[(1,2)][3])

In [ ]:
def eps_greedy(q_vals, eps, state):
    """
    Inputs:
        q_vals: q value tables
        eps: epsilon
        state: current state
    Outputs:
        random action with probability of eps; argmax Q(s, .) with probability of (1-eps)
    """
    # you might want to use random.random() to implement random exploration
    #   number of actions can be read off from len(q_vals[state])
    import random
    # YOUR CODE HERE

# test 1
dummy_q = defaultdict(lambda: np.array([0. for _ in range(env.action_space.n)]))
test_state = (0, 0)
dummy_q[test_state][0] = 10.
trials = 100000
sampled_actions = [
    int(eps_greedy(dummy_q, 0.3, test_state))
    for _ in range(trials)
]
freq = np.sum(np.array(sampled_actions) == 0) / trials
tgt_freq = 0.3 / env.action_space.n + 0.7
if np.isclose(freq, tgt_freq, atol=1e-2):
    print("Test1 passed")
else:
    print("Test1: Expected to select 0 with frequency %.2f but got %.2f" % (tgt_freq, freq))
    
# test 2
dummy_q = defaultdict(lambda: np.array([0. for _ in range(env.action_space.n)]))
test_state = (0, 0)
dummy_q[test_state][2] = 10.
trials = 100000
sampled_actions = [
    int(eps_greedy(dummy_q, 0.5, test_state))
    for _ in range(trials)
]
freq = np.sum(np.array(sampled_actions) == 2) / trials
tgt_freq = 0.5 / env.action_space.n + 0.5
if np.isclose(freq, tgt_freq, atol=1e-2):
    print("Test2 passed")
else:
    print("Test2: Expected to select 2 with frequency %.2f but got %.2f" % (tgt_freq, freq))

次は Q learning の更新則を実装します． $s, a, s', r$ という変化を観察した時，以下のように更新されます．

$$\textrm{target}(s') = R(s,a,s') + \gamma \max_{a'} Q_{\theta_k}(s',a')$$


$$Q_{k+1}(s,a) \leftarrow (1-\alpha) Q_k(s,a) + \alpha \left[ \textrm{target}(s') \right]$$

In [ ]:
def q_learning_update(gamma, alpha, q_vals, cur_state, action, next_state, reward):
    """
    Inputs:
        gamma: discount factor
        alpha: learning rate
        q_vals: q value table
        cur_state: current state
        action: action taken in current state
        next_state: next state results from taking `action` in `cur_state`
        reward: reward received from this transition
    
    Performs in-place update of q_vals table to implement one step of Q-learning
    """
    # YOUR CODE HERE

# testing your q_learning_update implementation
dummy_q = q_vals.copy()
test_state = (0, 0)
test_next_state = (0, 1)
dummy_q[test_state][0] = 10.
dummy_q[test_next_state][1] = 10.
q_learning_update(0.9, 0.1, dummy_q, test_state, 0, test_next_state, 1.1)
tgt = 10.01
if np.isclose(dummy_q[test_state][0], tgt,):
    print("Test passed")
else:
    print("Q(test_state, 0) is expected to be %.2f but got %.2f" % (tgt, dummy_q[test_state][0]))

In [ ]:
# now with the main components tested, we can put everything together to create a complete q learning agent

env = CrawlingRobotEnv() 
q_vals = defaultdict(lambda: np.array([0. for _ in range(env.action_space.n)]))
gamma = 0.9
alpha = 0.1
eps = 0.5
cur_state = env.reset()

def greedy_eval():
    """evaluate greedy policy w.r.t current q_vals"""
    test_env = CrawlingRobotEnv(horizon=np.inf)
    prev_state = test_env.reset()
    ret = 0.
    done = False
    H = 100
    for i in range(H):
        action = np.argmax(q_vals[prev_state])
        state, reward, done, info = test_env.step(action)
        ret += reward
        prev_state = state
    return ret / H

for itr in range(300000):
    # YOUR CODE HERE
    # Hint: use eps_greedy & q_learning_update
    
    if itr % 50000 == 0: # evaluation
        print("Itr %i # Average speed: %.2f" % (itr, greedy_eval()))

# at the end of learning your crawler should reach a speed of >= 3

学習が成功したら，ロボット制御を可視化して見ましょう！環境自体の dynamics を使うことなく，環境との相互干渉のみで学習を行なっているにも関わらず，うまく学習できていることがわかります．

In [ ]:
env = CrawlingRobotEnv(render=True, horizon=500)
prev_state = env.reset()
ret = 0.
done = False
while not done:
    action = np.argmax(q_vals[prev_state])
    state, reward, done, info = env.step(action)
    ret += reward
    prev_state = state

In [ ]:
# you can close the visualization GUI with the following method 
env.close_gui()